In [75]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Miten kommentti koostuu foorumilla?

In [ ]:
# Nesteen keskustelun linkki
r = requests.get("https://keskustelut.inderes.fi/t/harvia-foorumi-eli-haarumi-kansainvalista-kasvua-ja-hyvinvoinnin-megatrendeja/471/4")

# Jos status code = 200, vastaus saatiin
print(r)

# HTML:n jäsentäminen
soup = BeautifulSoup(r.content, 'html.parser')

# Haetaan tarvittavat elementit eli kommentit, aikaleima ja käyttäjä
comment = soup.find("div", {"class": "post"}).get_text(strip=True)
time = soup.find("time", {"class":"post-time"})
datetime = time['datetime']
creator = soup.find("span", {"class":"creator"})
name = creator.find("span", {"itemprop":"name"}).get_text(strip=True)

print(comment)
print(datetime)
print(name)

<Response [200]>
Hyvä Roasti tänään, vaikka olikin aika kevyt eikä hikeä tullu toimarilta. Harvialla todella puhtaat paperit ja tylsyys on hyvä sijoittamisessa, kuten Sammon kohdalla… Jatkoon… aloitan ostoprojektin huomenna.
2018-09-20T21:17:41Z
Kalle_P


## Kaikkien ketjun kommenttien hakeminen

In [76]:
# Listat kommenteille ja aikaleimoille

comments = []
dates = []
users = []

# Haetaan kaikki kommentit, joita on x kappaletta

for i in range(1,7997):
    r = requests.get(f"https://keskustelut.inderes.fi/t/harvia-foorumi-eli-haarumi-kansainvalista-kasvua-ja-hyvinvoinnin-megatrendeja/471/{i}")
    soup = BeautifulSoup(r.content, 'html.parser')

    # Kommentit
    comment = soup.find("div", {"class": "post"}).get_text(strip=True)
    comments.append(comment)

    # Aikaleimat
    time = soup.find("time", {"class":"post-time"})
    datetime = time['datetime']
    dates.append(datetime)
    
    # Käyttäjät
    creator = soup.find("span", {"class":"creator"})
    name = creator.find("span", {"itemprop":"name"}).get_text(strip=True)
    users.append(name)

In [101]:
# Siirretään kommentit dataframeihin

df_comments = pd.DataFrame({
    "datetime":dates,
    "user":users,
    "comment":comments
})

In [102]:
df_comments

,datetime,user,comment
0,2018-09-20T12:19:18Z,Verneri_Pulkkinen,Harvian toimari oli tänäänROASTissaja huomasin...
1,2018-09-20T12:43:56Z,timontti,"Harkinnassa, roast herätti mielenkiinnon ja la..."
2,2018-09-20T16:03:04Z,Markus14,"Karsastan tuota liikearvopommia, mikä yhtiöllä..."
3,2018-09-20T21:17:41Z,Kalle_P,"Hyvä Roasti tänään, vaikka olikin aika kevyt e..."
4,2018-09-21T05:25:08Z,Mikael_Rautanen,Vaikuttaa kyllä kaikin puolin laadukkaalta ja ...
...,...,...,...
7991,2025-02-24T16:19:50Z,kimsku,"Kiva kun pidetään ketjua aktiivisena, mutta re..."
7992,2025-02-24T16:44:17Z,Real,"onhan siinä HS:n kuvassa kiviä keskellä,HS-jut..."
7993,2025-02-26T17:44:22Z,Saapasjalkakissa,Mielenkiintoista tietoa pienen kannustimen vai...
7994,2025-02-27T05:14:06Z,eesau,Tässä vielä linkki lähteenä olevaan Ympäristöm...


In [ ]:
# Onko duplikaatteja?

df_comments.nunique()

datetime    7095
user         830
comment     7084
dtype: int64

In [ ]:
# Datassa on duplikaatteja, jotka näyttävät johtuvan siitä, että osa kommenteista on hävinnyt välistä
# Eli esim. kommenttia numero 10 ei ole enää
# Tämä mahdollisesti sekoittaa koodin ja luo duplikaatteja
# Riittävä taso minulle on vain poistaa duplikaatit

df_comments_unique = df_comments.drop_duplicates()

In [ ]:
# Nyt kommenttien määrä vastaa kommenttien määrää, joka foorumilla on datan keräys hetkellä näkyvissä

len(df_comments_unique)

7097

In [115]:
# Viedään kommentit talteen csv-tiedostoon

df_comments_unique.to_csv("comments.csv", index=False)